In [43]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
import binascii

key = RSA.generate(2048)
private_key = key.export_key()
public_key = key.publickey().export_key()

secret_key = get_random_bytes(16)
cipher_aes = AES.new(secret_key, AES.MODE_CBC)
iv = cipher_aes.iv

encrypted_private_key = cipher_aes.encrypt(pad(private_key, AES.block_size))

flag = b'hungabc'
cipher_rsa = PKCS1_OAEP.new(key.publickey())
ciphertext = cipher_rsa.encrypt(flag)

with open('secret_key.bin', 'wb') as f:
    f.write(secret_key)

with open('iv.bin', 'wb') as f:
    f.write(iv)

with open('private', 'wb') as f:
    f.write(encrypted_private_key)

with open('encrypt_flag', 'wb') as f:
    f.write(ciphertext)

with open('publickey.pem', 'wb') as f:
    f.write(public_key)

print(f"IV: {binascii.hexlify(iv).decode()}")
print(f"Encrypted Private Key (AES): {binascii.hexlify(encrypted_private_key).decode()}")
print(f"Encrypted Flag: {binascii.hexlify(ciphertext).decode()}")
print(f"Public Key: {public_key.decode()}")


IV: 2ba1a8528fd69749c7da41e098c317dd
Encrypted Private Key (AES): 264ffccf2712cdbdd809b4ec1beca5be075a658ca325760a7d9d872095947148da87ce30288dffed47167e5905b76fe981abaec28c3541f3fee1649f386b83fbd59d8b472ebc00d0fca8f712e54e1ea4cdad0a1c669177580c2c1ccc47e35e3d4b7e105b3df48500b49aea99919a2b021b21621c0604545785f650252786f82060a6bc5c73a889bfb65180118398503cd4332a96e1941f3292ed50ba636c74ce3ebe197c373f4775c75d308804dd9c5ad01e02da6c1edc859bcca8ae205b75bf59593dbb65a73652bd2eb584948ba2441534009c8d7ea8f7a246f633d366077e0212b03c94393a93712f2ba9c32f575684d45c90d0eb919b0f8eba1e36581cd786193df2f9a9a71a53b2ac0c4b73d1ab0f5d596e0d8a84d106814e9c734c4094f028536793af301bbd88263c96b41597cf56fca8661c43aaf3845323eaedc8b5bd11e2702bbfc53b9b5da9b7460e03ef64a9cb2b1925bab75a91cac00a36aca2a2a4f7b0a4528ff7d7ac86d34eee07e37acaecd6aa78a0060843941296d5e7179117b778dbab49b4df6d462f516d4415f119b9c9d0339530f0ebd740bcdf999ed19eb9be53a410e5226ca824a21a2fbe9b20c96629fb41f5a5e5d21092e17dbf39b3a9e10301e2730b10f847e5430a7c54e542

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Util.Padding import unpad
import binascii

# Step 1: Read the AES secret key and IV
with open('secret_key2.bin', 'rb') as f:
    secret_key = f.read()

with open('iv.bin', 'rb') as f:
    iv = f.read()

# Step 2: Decrypt the AES-encrypted RSA private key
with open('private', 'rb') as f:
    encrypted_private_key = f.read()

cipher_aes = AES.new(secret_key, AES.MODE_CBC, iv)
private_key_pem = unpad(cipher_aes.decrypt(encrypted_private_key), AES.block_size)

# Load the decrypted private key
private_key = RSA.import_key(private_key_pem)

# Step 3: Decrypt the encrypted flag using the private RSA key
with open('encrypt_flag2', 'rb') as f:
    encrypted_flag = f.read()

cipher_rsa = PKCS1_OAEP.new(private_key)
flag = cipher_rsa.decrypt(encrypted_flag)

# Print the decrypted flag
print(f"Decrypted Flag: {flag.decode()}")

Decrypted Flag: hungabc


In [ ]:
from PIL import Image
import binascii

def embed_file_into_image(image_path, file_path, output_image_path):
    # Đọc dữ liệu hình ảnh
    img = Image.open(image_path)
    img = img.convert("RGB")
    
    # Đọc dữ liệu file nhị phân
    with open(file_path, "rb") as f:
        file_data = f.read()
    
    # Kích thước hình ảnh
    width, height = img.size
    pixels = img.load()

    # Số byte cần giấu
    num_bytes = len(file_data)
    
    # Kiểm tra đủ dung lượng không
    if num_bytes > width * height * 3:
        raise ValueError("Hình ảnh không đủ lớn để chứa file.")
    
    # Giấu dữ liệu vào hình ảnh
    data_index = 0
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            if data_index < num_bytes:
                print(format(file_data[data_index], '08b'))
                r = (r & ~1) | ((file_data[data_index] >> 7) & 1)
                g = (g & ~1) | ((file_data[data_index] >> 6) & 1)
                b = (b & ~1) | ((file_data[data_index] >> 5) & 1)
                data_index += 1
            print(data_index)
            if data_index < num_bytes:
                print(file_data[data_index])
                r = (r & ~1) | ((file_data[data_index] >> 4) & 1)
                g = (g & ~1) | ((file_data[data_index] >> 3) & 1)
                b = (b & ~1) | ((file_data[data_index] >> 2) & 1)
                data_index += 1
            print(data_index)
            if data_index < num_bytes:
                print(file_data[data_index])
                r = (r & ~1) | ((file_data[data_index] >> 1) & 1)
                g = (g & ~1) | (file_data[data_index] & 1)
                data_index += 1
            print(data_index)
            pixels[x, y] = (r, g, b)
            if data_index >= num_bytes:
                break
        if data_index >= num_bytes:
            break
    
    # Lưu hình ảnh đã giấu dữ liệu
    img.save(output_image_path)

def extract_file_from_image(image_path, output_file_path):
    # Đọc dữ liệu hình ảnh
    img = Image.open(image_path)
    img = img.convert("RGB")
    
    # Kích thước hình ảnh
    width, height = img.size
    pixels = img.load()
    
    file_data = bytearray()
    
    data_index = 0
    byte = 0
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            byte = (byte << 1) | (r & 1)
            if data_index % 8 == 7:
                file_data.append(byte)
                byte = 0
            data_index += 1
            if data_index % 8 == 0:
                byte = (byte << 1) | (g & 1)
                data_index += 1
            if data_index % 8 == 1:
                byte = (byte << 1) | (b & 1)
                if data_index % 8 == 0:
                    file_data.append(byte)
                    byte = 0
            if data_index >= width * height * 3:
                break
        if data_index >= width * height * 3:
            break
    
    # Lưu file đã trích xuất
    with open(output_file_path, "wb") as f:
        f.write(file_data)

# Sử dụng
embed_file_into_image("RedBlackTree.png", "iv.bin", "output_image.png")
# extract_file_from_image("output_image.png", "iv2.bin")


In [41]:
def embed_file_into_elf(elf_path, file_path, output_elf_path):
    with open(elf_path, "rb") as elf_file:
        elf_data = elf_file.read()
    
    with open(file_path, "rb") as bin_file:
        bin_data = bin_file.read()
    
    # Tạo dữ liệu ELF mới với file nhị phân được nhúng
    embedded_elf_data = elf_data + b'\x00' * 4 + bin_data  # Thay đổi vị trí nhúng và độ dài theo nhu cầu

    with open(output_elf_path, "wb") as output_file:
        output_file.write(embedded_elf_data)

def extract_file_from_elf(elf_path, output_file_path):
    with open(elf_path, "rb") as elf_file:
        elf_data = elf_file.read()

    # Tìm vị trí của dữ liệu nhị phân được nhúng
    start_index = elf_data.rfind(b'\x00' * 4) + 4
    if start_index == -1:
        raise ValueError("Không tìm thấy dữ liệu nhị phân được nhúng trong ELF file.")
    
    # Trích xuất dữ liệu nhị phân
    bin_data = elf_data[start_index:]
    
    with open(output_file_path, "wb") as output_file:
        output_file.write(bin_data)

# Sử dụng
# embed_file_into_elf("simple.elf", "secret_key.bin", "output.elf")
extract_file_from_elf("output.elf", "secret_key2.bin")
